# ユニバーサルファンクション
NumPyのarrayの計算はとても早くも出来るし、とても遅くも出来る。早く計算するためのポイントは、NumPyのユニバーサルファンクション(ufuncs)を使用して計算することである。

## ループの遅さ
CPythonとして知られる、Pythonのデフォルトの実装において、いくつかの操作は非常に遅い。最近ではこの弱点を克服する以下のようないくつかの試みがある。

- PyPy
- Cython
- Numba

しかし、まだまだCPythonエンジンの実装ほど広まってはいない

ここでは例えば、arrayの値から逆数を計算したいと考える。一番単純なアプローチは次のようになるだろう。

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
print(values)
compute_reciprocals(values)

[6 1 4 4 8]


array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

この実装は自然に感じるかもしれないが、非常に遅い。IPythonの`%timeit`で時間を計測する。

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

2.56 s ± 232 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


100万の操作をするのに数秒かかっている。これはとても遅い。ここでのボトルネックは操作自身でなく、ループごとにCPythonが行わなければならない型チェックや関数のディスパッチである。

## UFuncsの導入
多くのタイプの操作に対して、NumPyはこの種の静的な型でコンパイル済みのルーチンへの便利なインターフェールを提供している。これはvectorized operationとして知られる。

以下の2つの操作の結果を比較する。

In [3]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


In [4]:
%timeit (1.0 / big_array)

2.99 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ufuncsはscalar対arrayだけでなく、scalar同士の計算も出来る。

また、多次元のarrayにおいても利用できる。

In [5]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

In [6]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

## NumPyのUFuncs
Ufuncsは2つのフレーバーが存在する。一つの入力への操作であるunary ufuncsと2つの入力への操作であるbinary ufuncs。

### Array arithmetic

In [7]:
x = np.arange(4)

print("x      =", x)
print("x + 5  =", x + 5)
print("x - 5  =", x - 5)
print("x * 2  =", x * 2)
print("x / 2  =", x / 2)
print("x // 2 =", x // 2)

print("-x     =", -x)
print("x ** 2 =", x ** 2)
print("x % 2  =", x % 2)

x      = [0 1 2 3]
x + 5  = [5 6 7 8]
x - 5  = [-5 -4 -3 -2]
x * 2  = [0 2 4 6]
x / 2  = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]
-x     = [ 0 -1 -2 -3]
x ** 2 = [0 1 4 9]
x % 2  = [0 1 0 1]


これらの演算子はNumPyにビルトインされた関数のラッパーである。例えば、`+`オペレータは`add`関数に対するラッパーである。

In [8]:
np.add(x, 2)

array([2, 3, 4, 5])

以下のテーブルはNumPyで実装されている算術演算子のリストである。

|Operator|Equivalent ufunc|Description|
|:---|:---|:---|
|`+`|np.add|Addition (e.g., 1 + 1 = 2)|
|`-`|np.subtract|Subtraction (e.g., 3 - 2 = 1)|
|`-`|np.negative|Unary negation (e.g., -2)|
|`*`|np.multiply|Multiplication| (e.g., 2 * 3 = 6)|
|`/`|np.divide|Division (e.g., 3 / 2 = 1.5)|
|`//`|np.floor_divide|Floor division (e.g., 3 // 2 = 1)|
|`**`|np.power|Exponentiation (e.g., 2 ** 3 = 8)|
|`%`|np.mod|Modulus/remainder (e.g., 9 % 4 = 1)|

## 絶対値
Pythonで組み込みの算術だけでなく、絶対値の関数も理解できる。

In [9]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

対応するNumPyのufuncは`np.absolute`である。これは`np.abs`のエイリアスからも利用できる。

In [10]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [11]:
np.abs(x)

array([2, 1, 0, 1, 2])

このufuncは複素数も扱うことができる。

In [12]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### 三角関数

In [13]:
theta = np.linspace(0, np.pi, 3)

In [14]:
print("theta = ", theta)

print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [15]:
x = [-1, 0, 1]

print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### べき乗と対数

In [16]:
x = [1, 2, 3]

print("x = ", x)
print("e^x = ", np.exp(x))
print("2^x = ", np.exp2(x))
print("3^x = ", np.power(3, x))

x =  [1, 2, 3]
e^x =  [ 2.71828183  7.3890561  20.08553692]
2^x =  [2. 4. 8.]
3^x =  [ 3  9 27]


In [17]:
x = [1, 2, 4, 10]

print("x        = ", x)
print("ln(x)    = ", np.log(x))
print("log2(x)  = ", np.log2(x))
print("log10(x) = ", np.log10(x))

x        =  [1, 2, 4, 10]
ln(x)    =  [0.         0.69314718 1.38629436 2.30258509]
log2(x)  =  [0.         1.         2.         3.32192809]
log10(x) =  [0.         0.30103    0.60205999 1.        ]


入力がとても小さい場合でも精度よく計算するために、特別な関数も用意されている。

In [18]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 = ", np.expm1(x))
print("log(1 + x) = ", np.log1p(x))

exp(x) - 1 =  [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) =  [0.         0.0009995  0.00995033 0.09531018]


### 特別なufuncs
NumPyには他にも様々なufuncsがある。

その他、素晴らしいufuncsが`scipy.special`にある。

In [19]:
from scipy import special

In [20]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [21]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


## 応用的なUfuncの特性

### 出力先の指定
大きな計算に対しては、計算結果をどこに保存するか指定出来れば便利である時がある。一時的なarrayを作成するより、直接書き込みたい場所に書き込めると便利になる。全てのufuncsは`out`引数をしようすることでこれが可能である。

In [22]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]


これはarrayのviewと共に利用することも出来る。例えば、計算結果を指定したarrayのそれぞれ別の要素へ書くことができる。

In [23]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


上記の代わりに、以下の通りに書いても結果は同じである。しかし、これは`2 ** x`の結果を保持する一時的なarrayを作成した後に、それらの値を`y`のarrayにコピーする操作となる。小規模な計算であれば、ほとんど変わらないが、大規模な計算になるとメモリの節約においても大きな差となる。

In [24]:
y[::2] = 2 ** x
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


### aggretates
arrayを特定の操作で一つの値に統合したいような場合には`reduce`メソッドが使える。

例えば、`reduce`を`add`のufuncで呼び出すと、arrayの中の全ての要素の和が返ってくる。

In [25]:
x = np.arange(1, 6)
np.add.reduce(x)

15

同様に、`reduce`を`multiply`で呼び出すと、全ての要素の積が返ってくる。

In [26]:
np.multiply.reduce(x)

120

計算の途中経過も保存しておきたいなら、代わりに`accumulate`が使用できる。

In [27]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [28]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

### outer products
`outer`メソッドを利用すると、2つの入力に対して全てのペアの結果を出力する。

In [29]:
x = np.arange(1, 6)
y = np.arange(2, 7)
print(x)
print(y)
np.multiply.outer(x, y)

[1 2 3 4 5]
[2 3 4 5 6]


array([[ 2,  3,  4,  5,  6],
       [ 4,  6,  8, 10, 12],
       [ 6,  9, 12, 15, 18],
       [ 8, 12, 16, 20, 24],
       [10, 15, 20, 25, 30]])